In [1]:
import torch
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [2]:
val_contexts, val_questions, val_answers = read_squad('preproc_datasets/dev-v2.0.json')

In [3]:
#test_context = val_contexts[5544]
#test_question = val_questions[5544]
#print('Answer:',val_answers[5544])

In [4]:
from transformers import BertTokenizerFast, BertModel

tokenizer = BertTokenizerFast.from_pretrained("dmis-lab/biobert-base-cased-v1.1", padding = True,
                                             truncation=True,add_special_tokens = True,
                                              model_max_length = 1000000000)

In [5]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
checkpoint = torch.load('checkpoint_squad_3epochs.pt',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

<All keys matched successfully>

In [7]:
val_predictions = []
for i in range(len(val_questions)):    
    inputs = tokenizer(val_questions[i], val_contexts[i], truncation=True, padding=True,max_length=500, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    # Get the most likely beginning of answer 
    answer_start = torch.argmax(answer_start_scores)  
    # Get the most likely end of answer 
    answer_end = torch.argmax(answer_end_scores) + 1  
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    #print(f"Question: {test_question}")
    #print(f"Answer: {answer}")
    val_predictions.append({'answer':answer,
                           'start':answer_start,
                           'end':answer_end})

In [ ]:
for key in inputs.keys():
    print(inputs[key].size())

In [15]:
val_predictions_clean = [{'answer':pred['answer'],
                         'start':int(pred['start']),
                         'end':int(pred['end'])} for pred in val_predictions]

In [19]:
import json
with open('pred_dev_3epochs.json', 'w') as fp:
    json.dump(val_predictions_clean, fp)